In [1]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import date
import datetime

# Generating the training data for the Heat and Diffusion Model

In [2]:
data_dir = "./LakeModeling/1D_HeatMixing_PGDL/output/"
depth_steps = 25

print(os.getcwd())

/home/robert/Projects/LakeModeling/1D_HeatMixing_PGDL/MCL


In [3]:
meterological_data_df = pd.read_csv("./../output/meteorology_input.csv")
meterological_data_df = meterological_data_df[1:] # considering everything from 2nd time step

num_time_steps = meterological_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
meterological_data_df = pd.DataFrame(np.repeat(meterological_data_df.values, depth_steps, axis=0), columns=meterological_data_df.columns)
meterological_data_df = pd.concat([depth_df, meterological_data_df], ignore_index=False, axis=1)
meterological_data_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2
0,1,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0
1,2,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0
2,3,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0
3,4,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0
4,5,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0
...,...,...,...,...,...,...,...,...,...,...,...
141570,21,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0
141571,22,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0
141572,23,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0
141573,24,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0


In [4]:
input_temp_df = pd.read_csv("./../output/temp_total04.csv")

flattened_inp_temp = input_temp_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
flattened_out_temp = input_temp_df.iloc[1:,1:].to_numpy().flatten() #this iloc is to remove the time column
time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values
data = {'time':time_stamp, 'input_temp':flattened_inp_temp, 'temp_total04':flattened_out_temp, 'depth':depth_list}
input_temp_df = pd.DataFrame(data=data)
input_temp_df 

,time,input_temp,temp_total04,depth
0,2009-06-04 01:00:00,16.283408,16.241692,1
1,2009-06-04 01:00:00,16.283408,16.247184,2
2,2009-06-04 01:00:00,16.287068,16.251763,3
3,2009-06-04 01:00:00,16.291257,16.251763,4
4,2009-06-04 01:00:00,16.278268,16.256606,5
...,...,...,...,...
141570,2012-07-31 23:00:00,8.537687,8.537921,21
141571,2012-07-31 23:00:00,7.466041,7.466186,22
141572,2012-07-31 23:00:00,6.396040,6.396125,23
141573,2012-07-31 23:00:00,5.326140,5.326179,24


In [5]:
depth_list.shape

(141575,)

In [6]:
final_df = meterological_data_df.merge(input_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04
0,1,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.241692
1,2,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.247184
2,3,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.287068,16.251763
3,4,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.291257,16.251763
4,5,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.278268,16.256606
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141570,21,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,8.537687,8.537921
141571,22,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,7.466041,7.466186
141572,23,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,6.396040,6.396125
141573,24,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,5.326140,5.326179


In [7]:
buoyancy_data_df = pd.read_csv("./../output/buoyancy.csv")
#Removing Nans
buoyancy_data_df['n2S-2_1'] = buoyancy_data_df['n2S-2_2']
buoyancy_data_df['n2S-2_25'] = buoyancy_data_df['n2S-2_24']

buoyancy_data_df = buoyancy_data_df[1:] # considering everything from 2nd time step

flattened_buoy = buoyancy_data_df.iloc[:,1:].to_numpy().flatten()
time_stamp = buoyancy_data_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'buoyancy':flattened_buoy, 'depth':depth_list}
buoyancy_data_df = pd.DataFrame(data=data)

buoyancy_data_df

,time,buoyancy,depth
0,2009-06-04 01:00:00,-0.000006,1
1,2009-06-04 01:00:00,-0.000006,2
2,2009-06-04 01:00:00,-0.000007,3
3,2009-06-04 01:00:00,0.000021,4
4,2009-06-04 01:00:00,0.000180,5
...,...,...,...
141570,2012-07-31 23:00:00,0.000637,21
141571,2012-07-31 23:00:00,0.000474,22
141572,2012-07-31 23:00:00,0.000307,23
141573,2012-07-31 23:00:00,0.000134,24


In [8]:
final_df = final_df.merge(buoyancy_data_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04,buoyancy
0,1,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.241692,-0.000006
1,2,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.247184,-0.000006
2,3,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.287068,16.251763,-0.000007
3,4,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.291257,16.251763,0.000021
4,5,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.278268,16.256606,0.000180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141570,21,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,8.537687,8.537921,0.000637
141571,22,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,7.466041,7.466186,0.000474
141572,23,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,6.396040,6.396125,0.000307
141573,24,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,5.326140,5.326179,0.000134


In [9]:
out_diffusivity_df = pd.read_csv("./../output/diff.csv")

out_diffusivity_df = out_diffusivity_df[1:] # considering everything from 2nd time step

flattened_diff = out_diffusivity_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_diffusivity_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'diffusivity':flattened_diff, 'depth':depth_list}
out_diffusivity_df = pd.DataFrame(data=data)

out_diffusivity_df

,time,diffusivity,depth
0,2009-06-04 01:00:00,0.000037,1
1,2009-06-04 01:00:00,0.000037,2
2,2009-06-04 01:00:00,0.000037,3
3,2009-06-04 01:00:00,0.000037,4
4,2009-06-04 01:00:00,0.000025,5
...,...,...,...
141570,2012-07-31 23:00:00,0.000014,21
141571,2012-07-31 23:00:00,0.000016,22
141572,2012-07-31 23:00:00,0.000020,23
141573,2012-07-31 23:00:00,0.000028,24


In [10]:
final_df = final_df.merge(out_diffusivity_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04,buoyancy,diffusivity
0,1,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.241692,-0.000006,0.000037
1,2,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.247184,-0.000006,0.000037
2,3,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.287068,16.251763,-0.000007,0.000037
3,4,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.291257,16.251763,0.000021,0.000037
4,5,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.278268,16.256606,0.000180,0.000025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141570,21,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,8.537687,8.537921,0.000637,0.000014
141571,22,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,7.466041,7.466186,0.000474,0.000016
141572,23,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,6.396040,6.396125,0.000307,0.000020
141573,24,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,5.326140,5.326179,0.000134,0.000028


In [11]:
out_temp_df = pd.read_csv("./../output/temp_heat00.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_heat00':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_heat00,depth
0,2009-06-04 01:00:00,16.283408,1
1,2009-06-04 01:00:00,16.283408,2
2,2009-06-04 01:00:00,16.287068,3
3,2009-06-04 01:00:00,16.291257,4
4,2009-06-04 01:00:00,16.278268,5
...,...,...,...
141570,2012-07-31 23:00:00,8.537687,21
141571,2012-07-31 23:00:00,7.466041,22
141572,2012-07-31 23:00:00,6.396040,23
141573,2012-07-31 23:00:00,5.326140,24


In [12]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04,buoyancy,diffusivity,temp_heat00
0,1,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.241692,-0.000006,0.000037,16.283408
1,2,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.247184,-0.000006,0.000037,16.283408
2,3,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.287068,16.251763,-0.000007,0.000037,16.287068
3,4,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.291257,16.251763,0.000021,0.000037,16.291257
4,5,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.278268,16.256606,0.000180,0.000025,16.278268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141570,21,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,8.537687,8.537921,0.000637,0.000014,8.537687
141571,22,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,7.466041,7.466186,0.000474,0.000016,7.466041
141572,23,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,6.396040,6.396125,0.000307,0.000020,6.396040
141573,24,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,5.326140,5.326179,0.000134,0.000028,5.326140


In [13]:
out_temp_df = pd.read_csv("./../output/temp_diff01.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_diff01':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_diff01,depth
0,2009-06-04 01:00:00,16.149422,1
1,2009-06-04 01:00:00,16.267964,2
2,2009-06-04 01:00:00,16.286071,3
3,2009-06-04 01:00:00,16.288695,4
4,2009-06-04 01:00:00,16.270156,5
...,...,...,...
141570,2012-07-31 23:00:00,8.537921,21
141571,2012-07-31 23:00:00,7.466186,22
141572,2012-07-31 23:00:00,6.396125,23
141573,2012-07-31 23:00:00,5.326179,24


In [14]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04,buoyancy,diffusivity,temp_heat00,temp_diff01
0,1,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.241692,-0.000006,0.000037,16.283408,16.149422
1,2,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.247184,-0.000006,0.000037,16.283408,16.267964
2,3,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.287068,16.251763,-0.000007,0.000037,16.287068,16.286071
3,4,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.291257,16.251763,0.000021,0.000037,16.291257,16.288695
4,5,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.278268,16.256606,0.000180,0.000025,16.278268,16.270156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141570,21,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,8.537687,8.537921,0.000637,0.000014,8.537687,8.537921
141571,22,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,7.466041,7.466186,0.000474,0.000016,7.466041,7.466186
141572,23,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,6.396040,6.396125,0.000307,0.000020,6.396040,6.396125
141573,24,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,5.326140,5.326179,0.000134,0.000028,5.326140,5.326179


In [15]:
datetime_list =[datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in final_df['time']]
day_of_year_list = [t.timetuple().tm_yday for t in datetime_list]
time_of_day_list = [t.hour for t in datetime_list]

In [16]:
final_df['day_of_year']=day_of_year_list
final_df['time_of_day']=time_of_day_list

In [17]:
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04,buoyancy,diffusivity,temp_heat00,temp_diff01,day_of_year,time_of_day
0,1,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.241692,-0.000006,0.000037,16.283408,16.149422,155,1
1,2,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.247184,-0.000006,0.000037,16.283408,16.267964,155,1
2,3,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.287068,16.251763,-0.000007,0.000037,16.287068,16.286071,155,1
3,4,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.291257,16.251763,0.000021,0.000037,16.291257,16.288695,155,1
4,5,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.278268,16.256606,0.000180,0.000025,16.278268,16.270156,155,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141570,21,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,8.537687,8.537921,0.000637,0.000014,8.537687,8.537921,213,23
141571,22,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,7.466041,7.466186,0.000474,0.000016,7.466041,7.466186,213,23
141572,23,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,6.396040,6.396125,0.000307,0.000020,6.396040,6.396125,213,23
141573,24,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,5.326140,5.326179,0.000134,0.000028,5.326140,5.326179,213,23


In [18]:
temp_mix02_df = pd.read_csv("./../output/temp_mix02.csv")

temp_mix02_df = temp_mix02_df[1:] # considering everything from 2nd time step

flattened_temp = temp_mix02_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_mix02_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_mix02':flattened_temp, 'depth':depth_list}
temp_mix02_df = pd.DataFrame(data=data)

temp_mix02_df

,time,temp_mix02,depth
0,2009-06-04 01:00:00,16.206994,1
1,2009-06-04 01:00:00,16.206994,2
2,2009-06-04 01:00:00,16.286071,3
3,2009-06-04 01:00:00,16.288695,4
4,2009-06-04 01:00:00,16.270156,5
...,...,...,...
141570,2012-07-31 23:00:00,8.537921,21
141571,2012-07-31 23:00:00,7.466186,22
141572,2012-07-31 23:00:00,6.396125,23
141573,2012-07-31 23:00:00,5.326179,24


In [19]:
final_df = final_df.merge(temp_mix02_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total04,buoyancy,diffusivity,temp_heat00,temp_diff01,day_of_year,time_of_day,temp_mix02
0,1,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.241692,-0.000006,0.000037,16.283408,16.149422,155,1,16.206994
1,2,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.283408,16.247184,-0.000006,0.000037,16.283408,16.267964,155,1,16.206994
2,3,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.287068,16.251763,-0.000007,0.000037,16.287068,16.286071,155,1,16.286071
3,4,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.291257,16.251763,0.000021,0.000037,16.291257,16.288695,155,1,16.288695
4,5,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,36000000.0,16.278268,16.256606,0.000180,0.000025,16.278268,16.270156,155,1,16.270156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141570,21,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,8.537687,8.537921,0.000637,0.000014,8.537687,8.537921,213,23,8.537921
141571,22,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,7.466041,7.466186,0.000474,0.000016,7.466041,7.466186,213,23,7.466186
141572,23,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,6.396040,6.396125,0.000307,0.000020,6.396040,6.396125,213,23,6.396125
141573,24,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,36000000.0,5.326140,5.326179,0.000134,0.000028,5.326140,5.326179,213,23,5.326179


In [20]:
temp_conv03_df = pd.read_csv("./../output/temp_conv03.csv")

temp_conv03_df = temp_conv03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_conv03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_conv03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_conv03':flattened_temp, 'depth':depth_list}
temp_conv03_df = pd.DataFrame(data=data)

temp_conv03_df

,time,temp_conv03,depth
0,2009-06-04 01:00:00,16.241692,1
1,2009-06-04 01:00:00,16.247184,2
2,2009-06-04 01:00:00,16.251763,3
3,2009-06-04 01:00:00,16.251763,4
4,2009-06-04 01:00:00,16.256606,5
...,...,...,...
141570,2012-07-31 23:00:00,8.537921,21
141571,2012-07-31 23:00:00,7.466186,22
141572,2012-07-31 23:00:00,6.396125,23
141573,2012-07-31 23:00:00,5.326179,24


In [21]:
final_df = final_df.merge(temp_conv03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,input_temp,temp_total04,buoyancy,diffusivity,temp_heat00,temp_diff01,day_of_year,time_of_day,temp_mix02,temp_conv03
0,1,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,16.283408,16.241692,-0.000006,0.000037,16.283408,16.149422,155,1,16.206994,16.241692
1,2,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,16.283408,16.247184,-0.000006,0.000037,16.283408,16.267964,155,1,16.206994,16.247184
2,3,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,16.287068,16.251763,-0.000007,0.000037,16.287068,16.286071,155,1,16.286071,16.251763
3,4,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,16.291257,16.251763,0.000021,0.000037,16.291257,16.288695,155,1,16.288695,16.251763
4,5,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,16.278268,16.256606,0.000180,0.000025,16.278268,16.270156,155,1,16.270156,16.256606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141570,21,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,...,8.537687,8.537921,0.000637,0.000014,8.537687,8.537921,213,23,8.537921,8.537921
141571,22,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,...,7.466041,7.466186,0.000474,0.000016,7.466041,7.466186,213,23,7.466186,7.466186
141572,23,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,...,6.396040,6.396125,0.000307,0.000020,6.396040,6.396125,213,23,6.396125,6.396125
141573,24,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,...,5.326140,5.326179,0.000134,0.000028,5.326140,5.326179,213,23,5.326179,5.326179


In [22]:
temp_obs_df = pd.read_csv("./../output/observed_temp.csv")

temp_obs_df = temp_obs_df[1:] # considering everything from 2nd time step

flattened_temp = temp_obs_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_obs_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'obs_temp':flattened_temp, 'depth':depth_list}
temp_obs_df = pd.DataFrame(data=data)

temp_obs_df

print(flattened_temp.shape)

print(time_stamp.shape)

print(depth_list.shape)

(141575,)
(141575,)
(141575,)


In [23]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,temp_total04,buoyancy,diffusivity,temp_heat00,temp_diff01,day_of_year,time_of_day,temp_mix02,temp_conv03,obs_temp
0,1,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,16.241692,-0.000006,0.000037,16.283408,16.149422,155,1,16.206994,16.241692,16.409
1,2,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,16.247184,-0.000006,0.000037,16.283408,16.267964,155,1,16.206994,16.247184,16.480
2,3,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,16.251763,-0.000007,0.000037,16.287068,16.286071,155,1,16.286071,16.251763,16.130
3,4,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,16.251763,0.000021,0.000037,16.291257,16.288695,155,1,16.288695,16.251763,15.827
4,5,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,16.256606,0.000180,0.000025,16.278268,16.270156,155,1,16.270156,16.256606,16.270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141570,21,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,...,8.537921,0.000637,0.000014,8.537687,8.537921,213,23,8.537921,8.537921,0.401
141571,22,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,...,7.466186,0.000474,0.000016,7.466041,7.466186,213,23,7.466186,7.466186,0.401
141572,23,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,...,6.396125,0.000307,0.000020,6.396040,6.396125,213,23,6.396125,6.396125,0.401
141573,24,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,...,5.326179,0.000134,0.000028,5.326140,5.326179,213,23,5.326179,5.326179,0.401


In [24]:

#temp_obs_df2 = pd.read_csv("./../output/observed_temp.csv")
#temp_obs_df = pd.read_csv("./../output/observed_temp.csv")


#print(flattened_inp_temp_obs.shape)

#temp_obs_df2 = temp_obs_df2[:-1] # considering everything from 2nd time step

#flattened_inp_temp_obs = temp_obs_df2.iloc[:,1:].to_numpy().flatten()

#time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values

#data = {'time':time_stamp, 'obs_input':flattened_inp_temp_obs, 'depth':depth_list}
#temp_obs_df2 = pd.DataFrame(data=data)

#temp_obs_df2


temp_obs_df = pd.read_csv("./../output/observed_temp.csv")

flattened_inp_temp = temp_obs_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
print(flattened_inp_temp.shape)

time_stamp = temp_obs_df['time'][1:].repeat(depth_steps).values
print(time_stamp.shape)

print(depth_list.shape)
data = {'time':time_stamp, 'input_obs':flattened_inp_temp,'depth':depth_list}
temp_obs_df = pd.DataFrame(data=data)
temp_obs_df 

(141575,)
(141575,)
(141575,)


,time,input_obs,depth
0,2009-06-04 01:00:00,16.350,1
1,2009-06-04 01:00:00,16.426,2
2,2009-06-04 01:00:00,16.088,3
3,2009-06-04 01:00:00,15.789,4
4,2009-06-04 01:00:00,16.240,5
...,...,...,...
141570,2012-07-31 23:00:00,0.401,21
141571,2012-07-31 23:00:00,0.401,22
141572,2012-07-31 23:00:00,0.401,23
141573,2012-07-31 23:00:00,0.401,24


In [25]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,buoyancy,diffusivity,temp_heat00,temp_diff01,day_of_year,time_of_day,temp_mix02,temp_conv03,obs_temp,input_obs
0,1,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,-0.000006,0.000037,16.283408,16.149422,155,1,16.206994,16.241692,16.409,16.350
1,2,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,-0.000006,0.000037,16.283408,16.267964,155,1,16.206994,16.247184,16.480,16.426
2,3,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,-0.000007,0.000037,16.287068,16.286071,155,1,16.286071,16.251763,16.130,16.088
3,4,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,0.000021,0.000037,16.291257,16.288695,155,1,16.288695,16.251763,15.827,15.789
4,5,2009-06-04 01:00:00,10.715021,695.937161,-56.268384,-23.291619,0.0,0.255324,1.085448,0.00229,...,0.000180,0.000025,16.278268,16.270156,155,1,16.270156,16.256606,16.270,16.240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141570,21,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,...,0.000637,0.000014,8.537687,8.537921,213,23,8.537921,8.537921,0.401,0.401
141571,22,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,...,0.000474,0.000016,7.466041,7.466186,213,23,7.466186,7.466186,0.401,0.401
141572,23,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,...,0.000307,0.000020,6.396040,6.396125,213,23,6.396125,6.396125,0.401,0.401
141573,24,2012-07-31 23:00:00,21.695001,832.055902,-93.493728,-12.091248,0.0,1.162606,1.049822,0.002174,...,0.000134,0.000028,5.326140,5.326179,213,23,5.326179,5.326179,0.401,0.401


In [26]:
final_df.to_csv("all_data_lake_modeling_in_time_wHeat_Oct.csv", index=False)